In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

In [9]:
data = pd.read_csv("Data/socialanxiety_llm_filtered_en.csv")

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  868 non-null    object
 1   Text    867 non-null    object
dtypes: object(2)
memory usage: 13.7+ KB


In [12]:
data = data[data['Text'].apply(lambda x: isinstance(x, str))]

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 867 entries, 0 to 867
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  867 non-null    object
 1   Text    867 non-null    object
dtypes: object(2)
memory usage: 20.3+ KB


In [14]:
data.head()

,Result,Text
0,yes,I can’t socialize or be around people unless I’m intoxicated in some way.
1,yes,"I've been struggling with social anxiety recently, especially around people who know me. Lately, I've noticed that I start itching intensely in these situations."
2,yes,"oh its okay, i can always suicide. its like having a 'i can always suicide if something goes wrong' kind of a mindset"
3,yes,Do people hate me or am I overthinking it?
4,yes,"I feel so bad since I'm usually organized, and I took it very serious. I know I'm only a human and we all make mistakes, I'll definitely not do it again that's for sure. So technically I should just stop thinking about it, but I can't. I'm so scared I'll get scolded in front of everyone tomorrow and idk what to do because I don't know how I'll react if that happens, although I don't think it'll happen."


In [15]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

texts = data['Text'].tolist()
translated_texts = batch_translate_text(texts)

data['Translated_Text'] = translated_texts

print(data)

C:\Users\ensar\Desktop\dogaldilisleme\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


    Result  \
0      yes   
1      yes   
2      yes   
3      yes   
4      yes   
..     ...   
863    yes   
864    yes   
865    yes   
866    yes   
867    yes   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Text  \
0                                                                                                                                                                                                                                                           

In [16]:
data

,Result,Text,Translated_Text
0,yes,I can’t socialize or be around people unless I’m intoxicated in some way.,Bir şekilde sarhoş olmadığım sürece sosyalleşemem ya da insanların etrafında olamam.
1,yes,"I've been struggling with social anxiety recently, especially around people who know me. Lately, I've noticed that I start itching intensely in these situations.","Son zamanlarda sosyal kaygılarla boğuşuyorum, özellikle de beni tanıyan insanların etrafında. Son zamanlarda, bu durumlarda yoğun bir şekilde kaşınmaya başladığımı fark ettim."
2,yes,"oh its okay, i can always suicide. its like having a 'i can always suicide if something goes wrong' kind of a mindset","Oh tamam, her zaman intihar edebilirim. 'Bir şeyler ters giderse her zaman intihar edebilirim' zihniyetine sahip olmak gibi bir şey."
3,yes,Do people hate me or am I overthinking it?,İnsanlar benden nefret mi ediyor yoksa ben mi aşırı düşünüyorum?
4,yes,"I feel so bad since I'm usually organized, and I took it very serious. I know I'm only a human and we all make mistakes, I'll definitely not do it again that's for sure. So technically I should just stop thinking about it, but I can't. I'm so scared I'll get scolded in front of everyone tomorrow and idk what to do because I don't know how I'll react if that happens, although I don't think it'll happen.","Genellikle organize olduğum için kendimi çok kötü hissediyorum ve çok ciddiye aldım. Biliyorum ben sadece bir insanım ve hepimiz hata yapıyoruz, kesinlikle bir daha yapmayacağım, bu kesin. Bu yüzden teknik olarak düşünmeyi bırakmalıyım, ama yapamam. Yarın herkesin önünde azarlanacağım ve ne yapacağımı bilmiyorum, çünkü böyle bir şey olmazsa nasıl tepki vereceğimi bilmiyorum."
...,...,...,...
863,yes,I haven’t interviewed in ages because I’ve been at my current job for a while and totally avoided looking for another one (due to interviewing and social anxiety)... I’m anxious for a whole week. SO anxious. ...I feel like I’ve destroyed this contact/relationship that I have by doing so poorly… I just had to talk about this because I can’t stop thinking about how bad I am at interviewing. I just hate the whole process. I’m so angry at myself and I feel depressed about the whole thing. I’m just cringing so much replaying back what I said. I’m so so angry at myself.,Uzun zamandır röportaj yapmadım çünkü bir süredir mevcut işimdeydim ve başka bir tane (görüşme ve sosyal kaygı nedeniyle) aramaktan tamamen kaçındım... Bütün bir hafta boyunca endişeliyim. Çok endişeliyim....Bu kadar kötü yaparak sahip olduğum bu teması / ilişkiyi mahvettiğimi hissediyorum... Sadece bunun hakkında konuşmak zorunda kaldım çünkü röportaj yaparken ne kadar kötü olduğumu düşünmeden duramıyorum. Sadece tüm bu süreçten nefret ediyorum.
864,yes,"When I watch videos and they look into the camera it feels like they can see me. Like I'm in a conversarion with them and they judge me, how I look or when I eat for an example. In public when I talk to someone I imagine the things they say like 'she can't even speak'","Videoları izlediğimde ve kameraya baktıklarında sanki beni görebiliyorlarmış gibi hissediyorum. Onlarla bir tartışma içindeymişim gibi ve beni yargılıyorlar, nasıl göründüğümü ya da bir örnek için yemek yediğimi. Biriyle konuştuğumda, 'konuşamıyor bile' gibi söyledikleri şeyleri hayal ediyorum."
865,yes,Haven’t been able to work or get groceries for weeks. Even driving feels too social for me now. I’m also self medicating a lot with alcohol and drugs,Haftalardır ne çalışabildim ne de bakkaliye alabildim. Araba kullanmak bile artık benim için çok sosyal geliyor. Ayrıca alkol ve uyuşturucu ile çok fazla ilaç kullanıyorum.
866,yes,"im scared to go out and do normal things, i get nervous when i have to talk to people, literally every morning before school my heart drops to my stomach for no reason","Dışarı çıkmaktan ve normal şeyler yapmaktan korkuyorum, insanlarla konuşmak zorunda kaldığımda gergin oluyorum, kelimenin tam anlamıyla her sabah okuldan ön

data.to_excel("panicdisorder_translated_to_tr.xlsx", index=False)